# DarkSkyAPI wrapper for Python 3.6

The DarkSkyAPI weather wrapper is powered by [DarkSky]("https://darksky.net/poweredby/") and provides an easy way to access weather details using Python 3.6.

## Client instance

First import the DarkSkyClient class from the DarkSkyAPI module. If you don't have an API key for the DarkSkyAPI yet, get one for free [here]("https://darksky.net/dev/register"). This will allow you to make a 1000 calls each day.

In [1]:
from DarkSkyAPI.DarkSkyAPI import DarkSkyClient
from secret import api_key

Next, create the client instance using the api_key as the first argument and a tuple containing the latitude and longitude of the location as the second argument. The third argument is optional and will set the units (Celsius/Fahrenheit). The unit options are as follows:

* auto: automatically select units based on geographic location
* ca: same as si, except that windSpeed and windGust are in kilometers per hour
* uk2: same as si, except that nearestStormDistance and visibility are in miles, and windSpeed and windGust in miles per hour.
* us: Imperial units
* si: International System of Units (default)

If no units are provided it will default to "si".

In [2]:
client = DarkSkyClient(api_key, (51.2279166, 5.8405417), units="si")

The client instance already holds the raw weather response and can be accessed by client.raw_data.

In [ ]:
client.raw_data

Additionally it also keeps track of the remaining api calls for the current day.

In [3]:
client.API_calls_remaining

997

## Current instance

To create the current instance, simply call the get_current method on client.

In [4]:
current = client.get_current()

All the data points of the current weather details are automatically set as attributes of the instance. This allows you to use the datapoints like attributes.

In [5]:
current.temperature

25.61

The weekday can be accessed by calling the weekday attribute on the current instance. This will return the full weekday name (in English). To return the short weekday name (i.e Mon, Tue), use the weekday_short attribute.

In [6]:
current.weekday

'Thursday'

In [7]:
current.weekday_short

'Thu'

These are the attributes available for the current instance.

In [8]:
for attr in dir(current):
    if "__" not in attr:
        print(attr)

_is_raining
_weekday
apparentTemperature
cloudCover
dewPoint
humidity
icon
is_raining
ozone
precipIntensity
precipProbability
pressure
summary
temperature
time
uvIndex
visibility
weekday
weekday_short
windBearing
windGust
windSpeed


## Daily and hourly instance

To create the daily and hourly instance, simply call the get_daily and get_hourly method on client.

In [9]:
daily = client.get_daily()
hourly = client.get_hourly()

To customize the amount of hours or days are returned, simply pass the n amount of hours/days as an int to the method.

In [ ]:
# Returns 6 days (including today)
daily = client.get_daily(6)
# Returns 24 hours (including current hour)
hourly = client.get_hourly(24)

The daily and hourly classes behave in the same way because they inherit from the same base class. Because of this, only the daily instance is documented. The terms hourly/daily and hour/day can be used interchangeably.

The forecast datapoints can be accessed in various ways. To get the data by individual days or hours you can either use the day/hour_n attribute or the data list of the forecast instance.

In [10]:
# Day attribute
daily.day_0['temperatureHigh']

25.95

In [11]:
# Daily data list
daily.data[0]['temperatureHigh']

25.95

Both instances also have every date point set as a property. These properties hold a list of single datapoint values.

In [12]:
daily.temperatureHigh

[25.95, 27.63, 28.21, 29.38, 29.51, 23.29, 25.15]

Alternatively, there are several methods you can use to get data collections of one or more datapoints. These methods work on both the daily and hourly instance. The methods currently available are:

* data_pair: Will return a list of value pair tuples containing firstly the datetime and secondly the value of the datapoint. This method accepts three arguments. The first argument is the datapoint (required). The second argument is the date_fmt parameter and will set the format of the datetime value (default - "%d-%m-%Y %H:%M"). The third argument is the graph argument, if set to True it wil return a graph-friendly dict of the datetime and values of the datapoint (default - False).
* data_single: Will return a list of single datapoint values. This method accepts three arguments. The first argument is the datapoint you wan the values of. The second argument is a boolean that will convert the datapoint to percentages if set to True (default: False). The third argument is a boolean that will convert the datapoint to a datetime string if set to True (default: False).
* data_combined: Will return a dict containing lists of datapoint values for each day/hour. This method accepts two arguments. The first is the list of datapoints. The second is the date_fmt incase the datapoint is time. If you don't provide a list of datapoints it will return all datapoints. 
* datetimes: Will return a list containing all the datetimes of the days/hours. This method accepts one argument which is the dateformat (default - "%d-%m-%Y %H:%M")



### Data pair method

In [13]:
# Data pair default date format and no graph
daily.data_pair('temperatureHigh')

[('12-07-2018 00:00', 25.95),
 ('13-07-2018 00:00', 27.63),
 ('14-07-2018 00:00', 28.21),
 ('15-07-2018 00:00', 29.38),
 ('16-07-2018 00:00', 29.51),
 ('17-07-2018 00:00', 23.29),
 ('18-07-2018 00:00', 25.15)]

In [14]:
# Data pair weekday names date_fmt
daily.data_pair('temperatureHigh', date_fmt="%A")

[('Thursday', 25.95),
 ('Friday', 27.63),
 ('Saturday', 28.21),
 ('Sunday', 29.38),
 ('Monday', 29.51),
 ('Tuesday', 23.29),
 ('Wednesday', 25.15)]

In [15]:
# Data pair graph
daily.data_pair('temperatureHigh', graph=True)

{'x': ['12-07-2018 00:00',
  '13-07-2018 00:00',
  '14-07-2018 00:00',
  '15-07-2018 00:00',
  '16-07-2018 00:00',
  '17-07-2018 00:00',
  '18-07-2018 00:00'],
 'y': [25.95, 27.63, 28.21, 29.38, 29.51, 23.29, 25.15]}

### Data single method

In [16]:
# Datapoint argument.
daily.data_single('temperatureHigh')

[25.95, 27.63, 28.21, 29.38, 29.51, 23.29, 25.15]

In [17]:
# Datapoint argument and to_percent argument set to True.
daily.data_single('precipProbability', to_percent=True)

[4, 0, 0, 0, 66, 86, 4]

In [18]:
# Datapoint argument and to_datetime argument set to True.
daily.data_single('precipIntensityMaxTime', to_datetime=True)

['12-07-2018 00:00',
 '13-07-2018 20:00',
 None,
 None,
 '16-07-2018 23:00',
 '17-07-2018 17:00',
 '18-07-2018 17:00']

### Data combined method

In [19]:
# Specified list of datapoints
daily.data_combined(['temperatureHigh', 'temperatureHighTime'], date_fmt="%H:%M")

{'temperatureHigh': [25.95, 27.63, 28.21, 29.38, 29.51, 23.29, 25.15],
 'temperatureHighTime': ['16:00',
  '17:00',
  '18:00',
  '16:00',
  '14:00',
  '15:00',
  '16:00']}

In [ ]:
# All data points
daily.data_combined()

### Datetimes method

In [20]:
# Default date format
daily.datetimes()

['12-07-2018 00:00',
 '13-07-2018 00:00',
 '14-07-2018 00:00',
 '15-07-2018 00:00',
 '16-07-2018 00:00',
 '17-07-2018 00:00',
 '18-07-2018 00:00']

In [21]:
# Weekday names date format (short)
daily.datetimes(date_fmt="%a")

['Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed']